In [1]:
%load_ext autoreload
%autoreload 2

import os
import wget
import zipfile
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from utils import *
from collections import Counter
from tqdm import tqdm
import time

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import the graphs from the saved files
G_brighkite_checkins = nx.read_gpickle(os.path.join('data', 'brightkite', 'brightkite_checkins_graph.gpickle'))
G_gowalla_checkins = nx.read_gpickle(os.path.join('data', 'gowalla', 'gowalla_checkins_graph.gpickle'))
G_foursquareEU_checkins = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareEU_checkins_graph.gpickle'))
G_foursquareIT_checkins = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareIT_checkins_graph.gpickle'))

G_brighkite_friends = nx.read_gpickle(os.path.join('data', 'brightkite', 'brightkite_friendships_graph.gpickle'))
G_gowalla_friends = nx.read_gpickle(os.path.join('data', 'gowalla', 'gowalla_friendships_graph.gpickle'))
G_foursquareEU_friends = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareEU_friendships_graph.gpickle'))
G_foursquareIT_friends = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareIT_friendships_graph.gpickle'))

# # open the dataframe object
# analysis_results = pd.read_pickle('analysis_results.pkl')
# analysis_results

The first thing that we want to do is very simple, create a random reference for each graph

In [4]:
analysis_results = pd.DataFrame(columns=['Graph', 'Number of Nodes', 'Number of Edges', 'Average Degree', 'Average Clustering Coefficient', 'log N', 'Average Shortest Path Length', 'betweenness centrality'], index=None)

checkins_graphs = [G_brighkite_checkins, G_gowalla_checkins, G_foursquareEU_checkins, G_foursquareIT_checkins]
friendships_graph = [G_brighkite_friends, G_gowalla_friends, G_foursquareIT_friends, G_foursquareEU_friends]

graphs_all = checkins_graphs + friendships_graph

# Original Graphs



In [ ]:
for graph in graphs_all:
    # add basic graph statistics
    analysis_results = analysis_results.append(
        {'Graph': graph.name, 
        'Number of Nodes': graph.number_of_nodes(), 
        'log N': np.log(graph.number_of_nodes()),
        'Number of Edges': graph.number_of_edges()}, 
        ignore_index=True)

    # add average degree
    print("Computing average degree for graph: ", graph.name)
    avg_deg = np.mean([d for n, d in graph.degree()])
    analysis_results.loc[analysis_results['Graph'] == graph.name, 'Average Degree'] = avg_deg

    # add average clustering coefficient
    print("Computing average clustering coefficient for graph: ", graph.name)
    avg_clustering = nx.average_clustering(graph)
    analysis_results.loc[analysis_results['Graph'] == graph.name, 'Average Clustering Coefficient'] = avg_clustering

    # add average shortest path length
    print("Computing average shortest path length for graph: ", graph.name)
    average_shortest_path_length = average_shortest_path(graph)
    analysis_results.loc[analysis_results['Graph'] == graph.name, 'Average Shortest Path Length'] = average_shortest_path_length

    # add betweenness centrality
    print("Computing betweenness centrality for graph: ", graph.name)
    betweenness_centrality = np.mean(list(betweenness_centrality_parallel(graph, 6).values()))
    analysis_results.loc[analysis_results['Graph'] == graph.name, 'betweenness centrality'] = betweenness_centrality
    print()


analysis_results
analysis_results.to_pickle('analysis_results.pkl')

# Random shit

In [6]:
analysis_results_erods = pd.DataFrame(columns=['Graph', 'Number of Nodes', 'Number of Edges', 'Average Degree', 'Average Clustering Coefficient', 'log N', 'Average Shortest Path Length', 'betweenness centrality'], index=None)

# read all the graphs gpickle files in the data/random/erdos folder. Then run the same analysis as before for this graphs

for graph in graphs_all:
    G = create_random_graphs(graph, "erods")

    # add the basic information to the dataframe
    analysis_results_erods = analysis_results_erods.append({
        'Graph': G.name,
        'Number of Nodes': G.number_of_nodes(),
        'Number of Edges': G.number_of_edges(),
        'log N': np.log(G.number_of_nodes())
    }, ignore_index=True)

    # compute the average degree and add it to the dataframe
    avg_deg = np.mean([d for n, d in G.degree()])
    analysis_results_erods.loc[analysis_results_erods['Graph'] == G.name, 'Average Degree'] = avg_deg

    # compute the average clustering coefficient and add it to the dataframe
    avg_clustering = nx.average_clustering(G)
    analysis_results_erods.loc[analysis_results_erods['Graph'] == G.name, 'Average Clustering Coefficient'] = avg_clustering

    # compute the average shortest path length and add it to the dataframe
    average_shortest_path_length = average_shortest_path(G)
    analysis_results_erods.loc[analysis_results_erods['Graph'] == G.name, 'Average Shortest Path Length'] = average_shortest_path_length

    # compute the betweenness centrality and add it to the dataframe
    betweenness_centrality = np.mean(list(betweenness_centrality_parallel(G, 6).values()))
    analysis_results_erods.loc[analysis_results_erods['Graph'] == G.name, 'betweenness centrality'] = betweenness_centrality

    # save memory
    del G

analysis_results_erods.to_pickle('analysis_results_erods.pkl')
analysis_results_erods


AttributeError: 'NoneType' object has no attribute 'name'

In [7]:
# do the same with the watts strogatz graphs

analysis_results_ws = pd.DataFrame(columns=['Graph', 'Number of Nodes', 'Number of Edges', 'Average Degree', 'Average Clustering Coefficient', 'log N', 'Average Shortest Path Length', 'betweenness centrality'], index=None)

for graph in graphs_all:
    G = create_random_graphs(graph, 'watts_strogatz', save=False)

    # add the basic information to the dataframe
    analysis_results_ws = analysis_results_ws.append({
        'Graph': G.name,
        'Number of Nodes': G.number_of_nodes(),
        'Number of Edges': G.number_of_edges(),
        'log N': np.log(G.number_of_nodes())
    }, ignore_index=True)

    # compute the average degree and add it to the dataframe
    avg_deg = np.mean([d for n, d in G.degree()])
    analysis_results_ws.loc[analysis_results_ws['Graph'] == G.name, 'Average Degree'] = avg_deg

    # compute the average clustering coefficient and add it to the dataframe
    avg_clustering = nx.average_clustering(G)
    analysis_results_ws.loc[analysis_results_ws['Graph'] == G.name, 'Average Clustering Coefficient'] = avg_clustering

    # compute the average shortest path length and add it to the dataframe
    average_shortest_path_length = average_shortest_path(G)
    analysis_results_ws.loc[analysis_results_ws['Graph'] == G.name, 'Average Shortest Path Length'] = average_shortest_path_length

    # compute the betweenness centrality and add it to the dataframe
    betweenness_centrality = np.mean(list(betweenness_centrality_parallel(G, 6).values()))
    analysis_results_ws.loc[analysis_results_ws['Graph'] == G.name, 'betweenness centrality'] = betweenness_centrality

    # save memory
    del G

analysis_results_ws.to_pickle('analysis_results_ws.pkl')
analysis_results_ws

	Number of edges in the original graph: 3663807
	Number of edges in the random graph: 3660219


UnboundLocalError: local variable 'G_copy' referenced before assignment

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
G = nx.watts_strogatz_graph(1000, 4, 0.1)
adj = nx.to_scipy_sparse_array(G)
# print info about the graph and the matrix
print("Number of nodes: ", G.number_of_nodes())
print("Number of edges: ", G.number_of_edges())
print("Average degree: ", np.mean([d for n, d in G.degree()]))
print("Average clustering coefficient: ", nx.average_clustering(G))
print("Average shortest path length: ", nx.average_shortest_path_length(G))

In [ ]:
import scipy.sparse as sp

# randomly swap edges, but keep the degree of each node the same (i.e. the degree sequence is preserved)
def random_swap_edges(adj, nswap=1, max_tries=100):
    # use numpy and scipy to speed up the process
    adj = sp.csr_matrix(adj)
    n, m = adj.shape 
    assert n == m # make sure the adjacency matrix is square
    adj_triu = sp.triu(adj) # only consider the upper triangular part of the adjacency matrix
    adj_tuple = sp.find(adj_triu) # get the indices and values of the non-zero elements
    adj_edges = np.array(list(zip(adj_tuple[0], adj_tuple[1]))) # get the edges
    adj_data = adj_tuple[2] # get the edge weights
    nnz = adj_edges.shape[0] # number of non-zero elements
    assert nnz == adj_data.shape[0] # make sure the number of edges and edge weights are the same
    for _ in range(nswap): # repeat nswap times
        # choose random edges to swap
        edge_idx = np.random.choice(nnz, size=2, replace=False) # choose two random edges
        edge1 = adj_edges[edge_idx[0]] # get the first edge
        edge2 = adj_edges[edge_idx[1]] # get the second edge
        # make sure the edges are not self-loops and not already connected
        if edge1[0] == edge2[0] or edge1[0] == edge2[1] or edge1[1] == edge2[0] or edge1[1] == edge2[1] or adj[edge1[0], edge2[1]] or adj[edge2[0], edge1[1]]: 
            continue # if the edges are self-loops or already connected, try again
        # swap the edges
        adj[edge1[0], edge1[1]] = 0 
        adj[edge2[0], edge2[1]] = 0     
        adj[edge1[0], edge2[1]] = 1
        adj[edge2[0], edge1[1]] = 1
        # update adj_edges and adj_data
        adj_edges[edge_idx[0]] = [edge1[0], edge2[1]]
        adj_edges[edge_idx[1]] = [edge2[0], edge1[1]]
        adj_data[edge_idx[0]] = 1
        adj_data[edge_idx[1]] = 1
    return adj

adj_swapped = random_swap_edges(adj, nswap=1)

In [ ]:
# create a new graph from the swapped adjacency matrix
G_swapped = nx.from_scipy_sparse_matrix(adj_swapped)
# print info about the graph and the matrix
print("Number of nodes: ", G_swapped.number_of_nodes())
print("Number of edges: ", G_swapped.number_of_edges())
print("Average degree: ", np.mean([d for n, d in G_swapped.degree()]))
print("Average clustering coefficient: ", nx.average_clustering(G_swapped))
print("Average shortest path length: ", nx.average_shortest_path_length(G_swapped))